<a href="https://colab.research.google.com/github/yogeshwaran23/Create-an-AI/blob/main/BRAIN_TUMER_CLASSIFICATIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import pandas

In [ ]:
## CONVERTING THE IMAGE INTO THE ARRAY




#PREPROCESSING OF TRAINING SET
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('DATASHEET/Training',
                                                 target_size = (150, 150),
                                                 batch_size = 100,
                                                 class_mode = 'categorical')

#PREPROCESSING OF TEST SET
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('DATASHEET/Testing',
                                            target_size = (150, 150),
                                            batch_size = 100,
                                            class_mode = 'categorical')



Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
#building an model of VGG16

from keras.applications import vgg16

vgg=vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(150,150,3))

#Freeze the layers
for layers in vgg.layers:
  layers.trainable=False


In [ ]:
from keras.src.layers.serialization import activation
from keras.src.layers import GlobalAveragePooling2D
def toplayers(Toplayer,number_class):
  Bottomlayer=Toplayer.output
  Bottomlayer=GlobalAveragePooling2D()(Bottomlayer)
  Bottomlayer=Dense(1024,activation='relu')(Bottomlayer)
  Bottomlayer=Dense(1024,activation='relu')(Bottomlayer)
  Bottomlayer=Dense(1024,activation='relu')(Bottomlayer)
  Bottomlayer=Dense(number_class,activation='softmax')(Bottomlayer)
  return Bottomlayer

In [ ]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D


FC_Head = toplayers(vgg, 4)

model = Model(inputs = vgg.input, outputs = FC_Head)

print(model.summary())

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0  

In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
58/58 [==============================] - 357s 6s/step - loss: 0.8347 - accuracy: 0.6625 - val_loss: 0.5925 - val_accuracy: 0.7750
Epoch 2/25
58/58 [==============================] - 344s 6s/step - loss: 0.4469 - accuracy: 0.8325 - val_loss: 0.5317 - val_accuracy: 0.7872
Epoch 3/25
58/58 [==============================] - 343s 6s/step - loss: 0.3875 - accuracy: 0.8484 - val_loss: 0.4671 - val_accuracy: 0.8284
Epoch 4/25
58/58 [==============================] - 346s 6s/step - loss: 0.3496 - accuracy: 0.8680 - val_loss: 0.5049 - val_accuracy: 0.8276
Epoch 5/25
58/58 [==============================] - 347s 6s/step - loss: 0.3196 - accuracy: 0.8834 - val_loss: 0.3705 - val_accuracy: 0.8551
Epoch 6/25
58/58 [==============================] - 347s 6s/step - loss: 0.2992 - accuracy: 0.8915 - val_loss: 0.4802 - val_accuracy: 0.8391
Epoch 7/25
58/58 [==============================] - 352s 6s/step - loss: 0.2845 - accuracy: 0.8953 - val_loss: 0.3777 - val_accuracy: 0.8574
Epoch 8/25
58

In [ ]:
#Predicting the result
import numpy as np
from keras.preprocessing import image
test_image=image.load_img('DATASHEET/predict/m_2.jpg',target_size=(150,150))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)

result=model.predict(test_image)
print(result)

training_set.class_indices

1/1 [==============================] - 0s 111ms/step
[[0.0000000e+00 1.0000000e+00 8.0983783e-29 0.0000000e+00]]


{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}

In [ ]:
if(result[0][0]==1):
    print('glioma')
elif(result[0][1]==1):
    print('meningioma')
elif(result[0][2]==1):
    print('notumor')
else:
    print('pituitary')



meningioma
